In [1]:
import re
import pandas as pd
import json
import argparse
from os.path import exists
from glob import glob
from os.path import isdir
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import warnings
import nltk
warnings.simplefilter(action='ignore')
import spacy
from nltk.corpus import wordnet
from nltk import pos_tag

In [2]:
import pandas as pd
import json

def load_dataset(file_name):
    df = []
    with open(file_name) as f:
        for i in f:
            i = json.loads(i)
            label = i['tags']
            label = label[0]
            
            if label == 'multi':
                continue
            
            df += [{'postText': i['postText'],
                    'targetParagraphs': i['targetParagraphs'],
                    'targetTitle': i['targetTitle'],
                    'targetDescription': i['targetDescription'],
                    'targetKeywords': i['targetKeywords'],
                    'spoiler': i['spoiler'],
                     'labels': (label == 'phrase' or label == 'phrases')}]

    return pd.DataFrame(df)




In [3]:
train_dataset = load_dataset('/content/train.jsonl')
validation_dataset = load_dataset('/content/validation.jsonl')

In [4]:
train_dataset

,postText,targetParagraphs,targetTitle,targetDescription,targetKeywords,spoiler,labels
0,"[Wes Welker Wanted Dinner With Tom Brady, But ...",[It’ll be just like old times this weekend for...,"Wes Welker Wanted Dinner With Tom Brady, But P...",It'll be just like old times this weekend for ...,"new england patriots, ricky doyle, top stories,",[how about that morning we go throw?],False
1,[NASA sets date for full recovery of ozone hole],[2070 is shaping up to be a great year for Mot...,Hole In Ozone Layer Expected To Make Full Reco...,2070 is shaping up to be a great year for Moth...,"ozone layer,ozone hole determined by weather,M...",[2070],True
2,[This is what makes employees happy -- and it'...,"[Despite common belief, money isn't the key to...",Intellectual Stimulation Trumps Money For Empl...,By: Chad Brooks \r\nPublished: 09/18/2013 06:4...,"employee happiness money,employee happiness in...",[intellectual stimulation],True
3,[The perfect way to cook rice so that it's per...,"[Boiling rice may seem simple, but there is a ...",Revealed: The perfect way to cook rice so that...,The question 'How does one cook rice properly?...,"Quora,users,share,perfect,way,cook,rice",[in a rice cooker],True
4,[What happens if your new AirPods get lost or ...,[One of the biggest surprise announcements at ...,Here's what happens if your Apple AirPods get ...,One of the first questions we had about Apple'...,None,[Apple says that if AirPods are lost or stolen...,False
...,...,...,...,...,...,...,...
2636,[If You See A Purple Butterfly Sticker At The ...,[If You See A Purple Butterfly Sticker At The ...,If You See A Purple Butterfly Sticker At The H...,Millie found out she was pregnant and she and ...,None,[one of her unborn twins had a condition calle...,False
2637,[Has Facebook's video explosion completely sha...,"[A long time ago in a galaxy far, far away...W...","Facebook Video Surging, But YouTube Still Offe...",.,"Facebook,web video,web video ads,YouTube",[it hasn’t necessarily taken the wind out of Y...,False
2638,[Cop Is Eating At A Chili's When Teen Hands Hi...,"[The Kansas City, Kansas Police Department are...",Cop is eating at Chili's when teen hands him f...,None,None,"[It read, ""Thanks for keeping us safe.""]",False
2639,[You need to see this Twitter account that pre...,"[What the HELL?!, 1. Unless you’re somewhere w...","WTF, It Looks Like This Twitter Account ""Predi...",,,[@beyoncefan666],True


In [5]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [6]:
nlp = spacy.load("en_core_web_sm")
if not nlp.vocab:
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [7]:
def alter_nouns_with_synonyms(text):
    doc = nlp(text)
    result = []
    for token in doc:
        if token.pos_ == 'NOUN':
            synonyms = []
            for syn in wordnet.synsets(token.text):
                for lemma in syn.lemmas():
                    synonyms.append(lemma.name())
            if synonyms:
                changed = False
                for syn in synonyms:
                    if syn != token.text:
                        changed = True
                        result.append(syn)
                        break
                if not changed:
                    result.append(token.text)
            else:
                result.append(token.text)
        else:
            result.append(token.text)
    return ' '.join(result)




In [8]:
df_result = []
for i in range(len(train_dataset)):
    post_text = train_dataset.iloc[i]['postText'][0]
    altered_text = alter_nouns_with_synonyms(post_text)
    temp_res = {'postText': [altered_text],
                'targetParagraphs': train_dataset.iloc[i]['targetParagraphs'],
                'targetTitle': train_dataset.iloc[i]['targetTitle'],
                'targetDescription': train_dataset.iloc[i]['targetDescription'],
                'targetKeywords': train_dataset.iloc[i]['targetKeywords'],
                'spoiler': train_dataset.iloc[i]['spoiler'],
                'labels': train_dataset.iloc[i]['labels']}
    df_result.append(temp_res)

altered_dataframe = pd.DataFrame(df_result, columns=['postText', 'targetParagraphs',
                                                     'targetTitle', 'targetDescription',
                                                     'targetKeywords', 'spoiler',
                                                     'labels'])

In [9]:
altered_dataframe



,postText,targetParagraphs,targetTitle,targetDescription,targetKeywords,spoiler,labels
0,"[Wes Welker Wanted Dinner With Tom Brady , But...",[It’ll be just like old times this weekend for...,"Wes Welker Wanted Dinner With Tom Brady, But P...",It'll be just like old times this weekend for ...,"new england patriots, ricky doyle, top stories,",[how about that morning we go throw?],False
1,[NASA sets day_of_the_month for full convalesc...,[2070 is shaping up to be a great year for Mot...,Hole In Ozone Layer Expected To Make Full Reco...,2070 is shaping up to be a great year for Moth...,"ozone layer,ozone hole determined by weather,M...",[2070],True
2,[This is what makes employee happy -- and it '...,"[Despite common belief, money isn't the key to...",Intellectual Stimulation Trumps Money For Empl...,By: Chad Brooks \r\nPublished: 09/18/2013 06:4...,"employee happiness money,employee happiness in...",[intellectual stimulation],True
3,[The perfect manner to cook Rice so that it 's...,"[Boiling rice may seem simple, but there is a ...",Revealed: The perfect way to cook rice so that...,The question 'How does one cook rice properly?...,"Quora,users,share,perfect,way,cook,rice",[in a rice cooker],True
4,[What happens if your new AirPods get lost or ...,[One of the biggest surprise announcements at ...,Here's what happens if your Apple AirPods get ...,One of the first questions we had about Apple'...,None,[Apple says that if AirPods are lost or stolen...,False
...,...,...,...,...,...,...,...
2636,[If You See A Purple Butterfly Sticker At The ...,[If You See A Purple Butterfly Sticker At The ...,If You See A Purple Butterfly Sticker At The H...,Millie found out she was pregnant and she and ...,None,[one of her unborn twins had a condition calle...,False
2637,[Has Facebook 's picture detonation completely...,"[A long time ago in a galaxy far, far away...W...","Facebook Video Surging, But YouTube Still Offe...",.,"Facebook,web video,web video ads,YouTube",[it hasn’t necessarily taken the wind out of Y...,False
2638,[bull Is Eating At A Chili 's When Teen Hands ...,"[The Kansas City, Kansas Police Department are...",Cop is eating at Chili's when teen hands him f...,None,None,"[It read, ""Thanks for keeping us safe.""]",False
2639,[You need to see this Twitter history that pre...,"[What the HELL?!, 1. Unless you’re somewhere w...","WTF, It Looks Like This Twitter Account ""Predi...",,,[@beyoncefan666],True


In [10]:
datasets = [train_dataset, altered_dataframe]
result = pd.concat(datasets)

In [11]:
result

,postText,targetParagraphs,targetTitle,targetDescription,targetKeywords,spoiler,labels
0,"[Wes Welker Wanted Dinner With Tom Brady, But ...",[It’ll be just like old times this weekend for...,"Wes Welker Wanted Dinner With Tom Brady, But P...",It'll be just like old times this weekend for ...,"new england patriots, ricky doyle, top stories,",[how about that morning we go throw?],False
1,[NASA sets date for full recovery of ozone hole],[2070 is shaping up to be a great year for Mot...,Hole In Ozone Layer Expected To Make Full Reco...,2070 is shaping up to be a great year for Moth...,"ozone layer,ozone hole determined by weather,M...",[2070],True
2,[This is what makes employees happy -- and it'...,"[Despite common belief, money isn't the key to...",Intellectual Stimulation Trumps Money For Empl...,By: Chad Brooks \r\nPublished: 09/18/2013 06:4...,"employee happiness money,employee happiness in...",[intellectual stimulation],True
3,[The perfect way to cook rice so that it's per...,"[Boiling rice may seem simple, but there is a ...",Revealed: The perfect way to cook rice so that...,The question 'How does one cook rice properly?...,"Quora,users,share,perfect,way,cook,rice",[in a rice cooker],True
4,[What happens if your new AirPods get lost or ...,[One of the biggest surprise announcements at ...,Here's what happens if your Apple AirPods get ...,One of the first questions we had about Apple'...,None,[Apple says that if AirPods are lost or stolen...,False
...,...,...,...,...,...,...,...
2636,[If You See A Purple Butterfly Sticker At The ...,[If You See A Purple Butterfly Sticker At The ...,If You See A Purple Butterfly Sticker At The H...,Millie found out she was pregnant and she and ...,None,[one of her unborn twins had a condition calle...,False
2637,[Has Facebook 's picture detonation completely...,"[A long time ago in a galaxy far, far away...W...","Facebook Video Surging, But YouTube Still Offe...",.,"Facebook,web video,web video ads,YouTube",[it hasn’t necessarily taken the wind out of Y...,False
2638,[bull Is Eating At A Chili 's When Teen Hands ...,"[The Kansas City, Kansas Police Department are...",Cop is eating at Chili's when teen hands him f...,None,None,"[It read, ""Thanks for keeping us safe.""]",False
2639,[You need to see this Twitter history that pre...,"[What the HELL?!, 1. Unless you’re somewhere w...","WTF, It Looks Like This Twitter Account ""Predi...",,,[@beyoncefan666],True


In [12]:
aug_data = result

In [13]:
aug_data

,postText,targetParagraphs,targetTitle,targetDescription,targetKeywords,spoiler,labels
0,"[Wes Welker Wanted Dinner With Tom Brady, But ...",[It’ll be just like old times this weekend for...,"Wes Welker Wanted Dinner With Tom Brady, But P...",It'll be just like old times this weekend for ...,"new england patriots, ricky doyle, top stories,",[how about that morning we go throw?],False
1,[NASA sets date for full recovery of ozone hole],[2070 is shaping up to be a great year for Mot...,Hole In Ozone Layer Expected To Make Full Reco...,2070 is shaping up to be a great year for Moth...,"ozone layer,ozone hole determined by weather,M...",[2070],True
2,[This is what makes employees happy -- and it'...,"[Despite common belief, money isn't the key to...",Intellectual Stimulation Trumps Money For Empl...,By: Chad Brooks \r\nPublished: 09/18/2013 06:4...,"employee happiness money,employee happiness in...",[intellectual stimulation],True
3,[The perfect way to cook rice so that it's per...,"[Boiling rice may seem simple, but there is a ...",Revealed: The perfect way to cook rice so that...,The question 'How does one cook rice properly?...,"Quora,users,share,perfect,way,cook,rice",[in a rice cooker],True
4,[What happens if your new AirPods get lost or ...,[One of the biggest surprise announcements at ...,Here's what happens if your Apple AirPods get ...,One of the first questions we had about Apple'...,None,[Apple says that if AirPods are lost or stolen...,False
...,...,...,...,...,...,...,...
2636,[If You See A Purple Butterfly Sticker At The ...,[If You See A Purple Butterfly Sticker At The ...,If You See A Purple Butterfly Sticker At The H...,Millie found out she was pregnant and she and ...,None,[one of her unborn twins had a condition calle...,False
2637,[Has Facebook 's picture detonation completely...,"[A long time ago in a galaxy far, far away...W...","Facebook Video Surging, But YouTube Still Offe...",.,"Facebook,web video,web video ads,YouTube",[it hasn’t necessarily taken the wind out of Y...,False
2638,[bull Is Eating At A Chili 's When Teen Hands ...,"[The Kansas City, Kansas Police Department are...",Cop is eating at Chili's when teen hands him f...,None,None,"[It read, ""Thanks for keeping us safe.""]",False
2639,[You need to see this Twitter history that pre...,"[What the HELL?!, 1. Unless you’re somewhere w...","WTF, It Looks Like This Twitter Account ""Predi...",,,[@beyoncefan666],True


In [14]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB
import re 
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [15]:
import nltk 
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
def lemmit(row):
  lemm_list = []
  for text in row['postText']:
    doc = nlp(text)
    for word in doc:
      if not word.is_punct and not word.is_stop:
        lemm_list.append(word.lemma_)
    break
  return ' '.join(lemm_list)


In [17]:
aug_data['postText'] = aug_data.apply(lemmit, axis=1)

In [18]:
aug_data

,postText,targetParagraphs,targetTitle,targetDescription,targetKeywords,spoiler,labels
0,Wes Welker want Dinner Tom Brady Patriots QB B...,[It’ll be just like old times this weekend for...,"Wes Welker Wanted Dinner With Tom Brady, But P...",It'll be just like old times this weekend for ...,"new england patriots, ricky doyle, top stories,",[how about that morning we go throw?],False
1,NASA set date recovery ozone hole,[2070 is shaping up to be a great year for Mot...,Hole In Ozone Layer Expected To Make Full Reco...,2070 is shaping up to be a great year for Moth...,"ozone layer,ozone hole determined by weather,M...",[2070],True
2,make employee happy paycheck,"[Despite common belief, money isn't the key to...",Intellectual Stimulation Trumps Money For Empl...,By: Chad Brooks \r\nPublished: 09/18/2013 06:4...,"employee happiness money,employee happiness in...",[intellectual stimulation],True
3,perfect way cook rice perfectly fluffy stick pan,"[Boiling rice may seem simple, but there is a ...",Revealed: The perfect way to cook rice so that...,The question 'How does one cook rice properly?...,"Quora,users,share,perfect,way,cook,rice",[in a rice cooker],True
4,happen new AirPods lose steal Apple,[One of the biggest surprise announcements at ...,Here's what happens if your Apple AirPods get ...,One of the first questions we had about Apple'...,None,[Apple says that if AirPods are lost or stolen...,False
...,...,...,...,...,...,...,...
2636,Purple Butterfly Sticker Hospital mean,[If You See A Purple Butterfly Sticker At The ...,If You See A Purple Butterfly Sticker At The H...,Millie found out she was pregnant and she and ...,None,[one of her unborn twins had a condition calle...,False
2637,Facebook picture detonation completely shake a...,"[A long time ago in a galaxy far, far away...W...","Facebook Video Surging, But YouTube Still Offe...",.,"Facebook,web video,web video ads,YouTube",[it hasn’t necessarily taken the wind out of Y...,False
2638,bull eat Chili Teen hand Folded Napkin walk aw...,"[The Kansas City, Kansas Police Department are...",Cop is eating at Chili's when teen hands him f...,None,None,"[It read, ""Thanks for keeping us safe.""]",False
2639,need Twitter history predict Beyoncé gestation,"[What the HELL?!, 1. Unless you’re somewhere w...","WTF, It Looks Like This Twitter Account ""Predi...",,,[@beyoncefan666],True


In [19]:

aug_data.dropna(inplace=True)

In [20]:


from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
aug_data['labels'] = label_encoder.fit_transform(aug_data['labels'])

In [21]:


from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split, KFold
X_train,X_test,y_train,y_test= train_test_split(aug_data.postText, aug_data.labels,test_size=0.3,random_state=42)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = TfidfVectorizer().fit(X_train)
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [23]:
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt


In [24]:

nb = MultinomialNB()
svm= SVC()
lr= LogisticRegression()

In [25]:
from sklearn.model_selection import RandomizedSearchCV
param_dist = {
    'C': np.logspace(-3, 3, 7),
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga'],
    'max_iter': [100, 500, 1000]
} 

In [26]:
kfold = KFold(n_splits=10,shuffle=True,random_state=0)
random_search = RandomizedSearchCV(lr, param_distributions=param_dist, n_iter=10, cv=kfold, scoring='f1_weighted', verbose=1, random_state=42)
random_search.fit(X_train,y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


RandomizedSearchCV(cv=KFold(n_splits=10, random_state=0, shuffle=True),
                   estimator=LogisticRegression(),
                   param_distributions={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                                        'max_iter': [100, 500, 1000],
                                        'penalty': ['l1', 'l2'],
                                        'solver': ['liblinear', 'saga']},
                   random_state=42, scoring='f1_weighted', verbose=1)

In [27]:
#for one vs one classifier

lr_ovo = OneVsOneClassifier(lr)
lr_ovo.fit(X_train, y_train)

svm_ovo = OneVsOneClassifier(svm)
svm_ovo.fit(X_train,y_train)


nb_ovo = OneVsOneClassifier(nb)
nb_ovo.fit(X_train, y_train)




OneVsOneClassifier(estimator=MultinomialNB())

In [28]:
validation_dataset = load_dataset('/content/validation.jsonl')
validation_dataset['postText'] = validation_dataset.apply(lemmit, axis=1)
validation_dataset['labels'] = label_encoder.transform(validation_dataset['labels'])
X_val = validation_dataset['postText']
Y_val = validation_dataset['labels']
X_val = vectorizer.transform(X_val)
lr_ovo_pred = lr_ovo.predict(X_val)

In [29]:
validation_dataset.shape

(657, 7)

In [30]:
lr_best = random_search.best_estimator_
lr_best_predict = lr_best.predict(X_val)

In [31]:
print("Tuned logistic regression ")
print(classification_report(Y_val,lr_best_predict))
cm_lr = confusion_matrix(Y_val, lr_best_predict)
tn_best_lr, fp_best_lr, fn_best_lr, tp_best_lr = cm_lr.ravel()
print("Tuned Logistic Regression:")
print(f"TP: {tp_best_lr}, TN: {tn_best_lr}, FP: {fp_best_lr}, FN: {fn_best_lr}")



Tuned logistic regression 
              precision    recall  f1-score   support

           0       0.62      0.54      0.58       322
           1       0.61      0.67      0.64       335

    accuracy                           0.61       657
   macro avg       0.61      0.61      0.61       657
weighted avg       0.61      0.61      0.61       657

Tuned Logistic Regression:
TP: 226, TN: 175, FP: 147, FN: 109


In [32]:
print("one vs one logistic regression ")
print(classification_report(Y_val,lr_ovo_pred))

cm_lr_ovo = confusion_matrix(Y_val, lr_ovo_pred)
tn_lr_ovo, fp_lr_ovo, fn_lr_ovo, tp_lr_ovo = cm_lr_ovo.ravel()
print("One vs One Logistic Regression:")
print(f"TP: {tp_lr_ovo}, TN: {tn_lr_ovo}, FP: {fp_lr_ovo}, FN: {fn_lr_ovo}")


one vs one logistic regression 
              precision    recall  f1-score   support

           0       0.63      0.46      0.53       322
           1       0.59      0.73      0.65       335

    accuracy                           0.60       657
   macro avg       0.61      0.60      0.59       657
weighted avg       0.61      0.60      0.59       657

One vs One Logistic Regression:
TP: 246, TN: 149, FP: 173, FN: 89


In [33]:
val_svm_ovo_pred = svm_ovo.predict(X_val)
print("one vs one SVM ")
print(classification_report(Y_val,val_svm_ovo_pred))

cm_svm_ovo = confusion_matrix(Y_val, val_svm_ovo_pred)
tn_svm_ovo, fp_svm_ovo, fn_svm_ovo, tp_svm_ovo = cm_svm_ovo.ravel()
print(f"TP: {tp_svm_ovo}, TN: {tn_svm_ovo}, FP: {fp_svm_ovo}, FN: {fn_svm_ovo}")


one vs one SVM 
              precision    recall  f1-score   support

           0       0.62      0.47      0.53       322
           1       0.59      0.73      0.65       335

    accuracy                           0.60       657
   macro avg       0.61      0.60      0.59       657
weighted avg       0.61      0.60      0.59       657

TP: 245, TN: 150, FP: 172, FN: 90


In [34]:
from scipy.stats import uniform, randint
param_dist_svm = {'C': uniform(loc=1, scale=10),
              'gamma': uniform(loc=0.01, scale=0.1),
              'kernel': ['linear', 'rbf']}
random_search_svm = RandomizedSearchCV(SVC(), param_dist_svm, refit=True, verbose=2, scoring='f1_weighted', cv=kfold, n_iter=20)
random_search_svm.fit(X_train, y_train)
              

Fitting 10 folds for each of 20 candidates, totalling 200 fits
[CV] END C=2.905171265175778, gamma=0.07121233727703521, kernel=linear; total time=   0.4s
[CV] END C=2.905171265175778, gamma=0.07121233727703521, kernel=linear; total time=   0.5s
[CV] END C=2.905171265175778, gamma=0.07121233727703521, kernel=linear; total time=   0.5s
[CV] END C=2.905171265175778, gamma=0.07121233727703521, kernel=linear; total time=   0.5s
[CV] END C=2.905171265175778, gamma=0.07121233727703521, kernel=linear; total time=   0.5s
[CV] END C=2.905171265175778, gamma=0.07121233727703521, kernel=linear; total time=   0.5s
[CV] END C=2.905171265175778, gamma=0.07121233727703521, kernel=linear; total time=   0.5s
[CV] END C=2.905171265175778, gamma=0.07121233727703521, kernel=linear; total time=   0.5s
[CV] END C=2.905171265175778, gamma=0.07121233727703521, kernel=linear; total time=   0.5s
[CV] END C=2.905171265175778, gamma=0.07121233727703521, kernel=linear; total time=   0.5s
[CV] END C=2.95293727721150

RandomizedSearchCV(cv=KFold(n_splits=10, random_state=0, shuffle=True),
                   estimator=SVC(), n_iter=20,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f59f6918880>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f59ff50f1f0>,
                                        'kernel': ['linear', 'rbf']},
                   scoring='f1_weighted', verbose=2)

In [35]:
best_params_svm = random_search_svm.best_params_

In [36]:
svm_best = SVC(**best_params_svm)
svm_best.fit(X_train, y_train)
val_svm_best_pred = svm_best.predict(X_val)
print("tuned svm")
print(classification_report(Y_val, val_svm_best_pred))

cm_svm_best = confusion_matrix(Y_val, val_svm_best_pred)
tn_svm_best, fp_svm_best, fn_svm_best, tp_svm_best = cm_svm_best.ravel()
print(f"TP: {tp_svm_best}, TN: {tn_svm_best}, FP: {fp_svm_best}, FN: {fn_svm_best}")


tuned svm
              precision    recall  f1-score   support

           0       0.60      0.53      0.56       322
           1       0.59      0.65      0.62       335

    accuracy                           0.59       657
   macro avg       0.59      0.59      0.59       657
weighted avg       0.59      0.59      0.59       657

TP: 218, TN: 172, FP: 150, FN: 117


In [37]:
len(val_svm_best_pred)

657

In [38]:
val_nb_ovo_pred = nb_ovo.predict(X_val)
print("one vs one nb ")
print(classification_report(Y_val,val_nb_ovo_pred))

cm_nb_ovo = confusion_matrix(Y_val, val_nb_ovo_pred)
tn_nb_ovo, fp_nb_ovo, fn_nb_ovo, tp_nb_ovo = cm_nb_ovo.ravel()
print(f"TP: {tp_nb_ovo}, TN: {tn_nb_ovo}, FP: {fp_nb_ovo}, FN: {fn_nb_ovo}")


one vs one nb 
              precision    recall  f1-score   support

           0       0.59      0.44      0.51       322
           1       0.57      0.71      0.63       335

    accuracy                           0.58       657
   macro avg       0.58      0.58      0.57       657
weighted avg       0.58      0.58      0.57       657

TP: 238, TN: 142, FP: 180, FN: 97


In [39]:
from scipy.stats import uniform
param_dist_nb = {'alpha': uniform(loc=0.1, scale=0.9)}
random_search_nb = RandomizedSearchCV(nb, param_dist_nb, n_iter=20, scoring='f1_weighted', cv=kfold)
random_search_nb.fit(X_train, y_train)

RandomizedSearchCV(cv=KFold(n_splits=10, random_state=0, shuffle=True),
                   estimator=MultinomialNB(), n_iter=20,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f5a00ee12d0>},
                   scoring='f1_weighted')

In [40]:
best_params_nb = random_search_nb.best_params_

nb_best = MultinomialNB(alpha=best_params_nb['alpha'])
nb_best.fit(X_train, y_train)

MultinomialNB(alpha=0.10859841568127998)

In [41]:
val_nb_best_pred = nb_best.predict(X_val)
print("Multinomial NB with hyperparameters")
print(classification_report(Y_val, val_nb_best_pred))

cm_nb_best = confusion_matrix(Y_val, val_nb_best_pred)
tn_nb_best, fp_nb_best, fn_nb_best, tp_nb_best = cm_nb_best.ravel()
print(f"TP: {tp_nb_best}, TN: {tn_nb_best}, FP: {fp_nb_best}, FN: {fn_nb_best}")


Multinomial NB with hyperparameters
              precision    recall  f1-score   support

           0       0.58      0.47      0.52       322
           1       0.57      0.67      0.61       335

    accuracy                           0.57       657
   macro avg       0.57      0.57      0.57       657
weighted avg       0.57      0.57      0.57       657

TP: 224, TN: 152, FP: 170, FN: 111
